In [2]:
import numpy as np
import pandas as pd
import re, string

### 유사도 테스트 해보기

In [3]:
df = pd.read_csv('../static/data/movie.csv')
df.code = df.code.astype(str)
df.fillna('', inplace=True)
print(df.shape)
df.head(1)

(11270, 17)


,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis,m_type,m_kind,m_genre,m_nation,m_time,m_rated,morphs
0,20231523,그 여름 (The Summer),애니메이션(2),"장편 | 예술,독립 영화 | 애니메이션, 드라마, 멜로/로맨스 | 60분 57초 |...",2023-06-07,2023년,한지원,윤아영 | 송하림,https://www.kobis.or.kr/common/mast/movie/2023...,"""서로의 몸은 차라리 꽃잎과 물결에 가까웠다""\r\n갈색 눈동자를 가진 평범한 학생...",장편,"예술,독립 영화",애니메이션 드라마 멜로/로맨스,한국,60분 57초,12세이상관람가,서로 몸 차라리 꽃잎 물결 가까웠다 갈색 눈동자 가진 평범한 학생 이경 여름 햇살 ...


##### 1. 형태소 분석 + 제목 + 장르 + 감독 + 배우 로 합쳐서 코사인 유사도 구해보기

In [4]:
def clean_text(texts):
    clean_text = []
    for text in texts:
        
        text = re.sub(r'\([^)]*\)', '', text) 
        text = re.sub('['+string.punctuation+']', ' ', text).lower()
        clean_text.append(text)
    
    return clean_text

In [5]:
# df['total'] = df.morphs + (' ' + df.title) + (' ' + df.m_genre) * 3 + (' ' + df.movie_director) * 2 + (' ' + df.star_actor) * 2
df['re_director'] = clean_text(df.movie_director)
df['re_actor'] = clean_text(df.star_actor)

df['total'] = df.morphs + (' ' + df.title) + (' ' + df.m_genre) * 3 + \
                (' ' + df.re_director) * 2 + (' ' + df.re_actor) * 2

In [6]:
df.head(1)

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis,m_type,m_kind,m_genre,m_nation,m_time,m_rated,morphs,re_director,re_actor,total
0,20231523,그 여름 (The Summer),애니메이션(2),"장편 | 예술,독립 영화 | 애니메이션, 드라마, 멜로/로맨스 | 60분 57초 |...",2023-06-07,2023년,한지원,윤아영 | 송하림,https://www.kobis.or.kr/common/mast/movie/2023...,"""서로의 몸은 차라리 꽃잎과 물결에 가까웠다""\r\n갈색 눈동자를 가진 평범한 학생...",장편,"예술,독립 영화",애니메이션 드라마 멜로/로맨스,한국,60분 57초,12세이상관람가,서로 몸 차라리 꽃잎 물결 가까웠다 갈색 눈동자 가진 평범한 학생 이경 여름 햇살 ...,한지원,윤아영 송하림,서로 몸 차라리 꽃잎 물결 가까웠다 갈색 눈동자 가진 평범한 학생 이경 여름 햇살 ...


In [7]:
# TfidfVectorizer 사용
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer(stop_words='english')
total_tv = tvect.fit_transform(df.total)
total_tv.shape

(11270, 83032)

In [8]:
# CountVectorizer 사용
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words='english')
total_cv = cvect.fit_transform(df.total)
total_cv.shape

(11270, 83032)

In [9]:
# index화 code값 매칭
indices = pd.Series(df.index, index=df.code)

In [10]:
# TfidfVectorizer 의 코사인 유사도
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(total_tv)
cosine_sim.shape

(11270, 11270)

In [11]:
# CountVectorizer 의 코사인 유사도
cosine_sim_cv = cosine_similarity(total_cv)
cosine_sim_cv.shape

(11270, 11270)

In [12]:
# 파일로 저장해서 사용하는 더 빠름
import joblib

joblib.dump(cosine_sim_cv, '../static/data/movie_cosine_sim.sim')

['../static/data/movie_cosine_sim.sim']

In [13]:
def get_recommendation(code, cos_sim):
    index = indices[code]
    sim_scores = pd.Series(cos_sim[index])
    movie_indices = sim_scores.sort_values(ascending=False).head(11).tail(10).index
    return df.code.iloc[movie_indices], df.title.iloc[movie_indices]

In [14]:
# TfidfVectorizer 의 코사인 유사도 %
from sklearn.metrics.pairwise import linear_kernel
cosine_sim_total = linear_kernel(total_tv, total_tv)
cosine_sim_total.shape

(11270, 11270)

In [15]:
# CountVectorizer 의 코사인 유사도 %
from sklearn.metrics.pairwise import linear_kernel
cosine_sim_total_cv = linear_kernel(total_cv, total_cv)
cosine_sim_total_cv.shape

(11270, 11270)

In [16]:
df.code

0        20231523
1        20232506
2        20232003
3        20232570
4        20226897
           ...   
11265    20203281
11266    20188682
11267    20020049
11268    19828097
11269    20190105
Name: code, Length: 11270, dtype: object

In [17]:
code = "20188682"

In [18]:
# TfidfVectorizer 의 유사 영화
codes, titles = get_recommendation(code, cosine_sim_cv)
codes, titles

(5684     20205902
 6488     20200401
 1270     20219246
 73       20231043
 2639     20216583
 10823    20208314
 2762     20206806
 10678    20192162
 6241     20217031
 2119     20190532
 Name: code, dtype: object,
 5684                            신과 나: 100일간의 거래 (Homestay)
 6488                                 오! 슬프도다 (Alas for Me)
 1270     나츠메 우인장: 이시오코시와 수상한 방문자 (Natsume Yujin-cho Ish...
 73       골목길 0번지 인격상담소 (Personality Counseling Center a...
 2639                                 루시드 드림 (Lucid Dreams)
 10823                                         확대 (Blow-Up)
 2762                               리멤버: 기억의 살인자 (Remember)
 10678        혈관음 (The Bold, the Corrupt and the Beautiful)
 6241                                         어브로드 (ABROAD)
 2119                          드래곤 빌리지 (Family of Mischief)
 Name: title, dtype: object)

In [19]:
# CountVectorizer 의 유사 영화
get_recommendation(code, cosine_sim)

(10631    20230138
 611      20202442
 5684     20205902
 5372     20218924
 682      20195802
 7020     20214813
 4288     20231620
 5410     20218234
 52       20231009
 2119     20190532
 Name: code, dtype: object,
 10631                                      하룻밤 (One Night)
 611                         극장판 시로바코 (SHIROBAKO The Movie)
 5684                            신과 나: 100일간의 거래 (Homestay)
 5372                        살인봉황 (The Great Tang Monsters)
 682      걸즈 앤 판처 제 63회 전차도 전국 고교생 대회 (Girls und Panzer ...
 7020                                음악 (ON-GAKU:Our Sound)
 4288              바깥 나라의 소녀 (The Girl from the Other Side)
 5410                               산해거수 (KongFu Master Su)
 52                 거울 속 외딴 성 (Lonely Castle in the Mirror)
 2119                          드래곤 빌리지 (Family of Mischief)
 Name: title, dtype: object)

In [20]:
# TfidfVectorizer 의 유사 영화 %
sim_scores = pd.Series(cosine_sim[indices[code]])
score = sim_scores.sort_values(ascending=False).head(10).tail(9)
score

10631    0.179551
611      0.175426
5684     0.145988
5372     0.136572
682      0.124317
7020     0.123337
4288     0.122053
5410     0.113170
52       0.112670
dtype: float64

In [21]:
# CountVectorizer 의 유사 영화 %
sim_scores = pd.Series(cosine_sim_cv[indices[code]])
score = sim_scores.sort_values(ascending=False).head(10).tail(9)
score

5684     0.320627
6488     0.318932
1270     0.316054
73       0.300364
2639     0.292306
10823    0.290119
2762     0.282865
10678    0.281272
6241     0.280646
dtype: float64

In [22]:
codes

5684     20205902
6488     20200401
1270     20219246
73       20231043
2639     20216583
10823    20208314
2762     20206806
10678    20192162
6241     20217031
2119     20190532
Name: code, dtype: object

In [23]:
df[df.code.isin(codes)][['title']]

,title
73,골목길 0번지 인격상담소 (Personality Counseling Center a...
1270,나츠메 우인장: 이시오코시와 수상한 방문자 (Natsume Yujin-cho Ish...
2119,드래곤 빌리지 (Family of Mischief)
2639,루시드 드림 (Lucid Dreams)
2762,리멤버: 기억의 살인자 (Remember)
5684,신과 나: 100일간의 거래 (Homestay)
6241,어브로드 (ABROAD)
6488,오! 슬프도다 (Alas for Me)
10678,"혈관음 (The Bold, the Corrupt and the Beautiful)"
10823,확대 (Blow-Up)


### 결론 : CountVectorizer 로 유사도 구한 것이 %가 더 좋았음. CountVectorizer로 추천 시스템을 제작